<a href="https://colab.research.google.com/github/priyu9-star/BudgetWise-AI-based-Expense-Forecasting-Tool-Batch-6-Team-C-/blob/main/AIExpenseTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Run
!pip install -q flask pyngrok plotly pandas matplotlib pyarrow

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Personal_Finance_Dataset.csv to Personal_Finance_Dataset (1).csv


In [ ]:
import re
import json
from pathlib import Path
from typing import Optional
import pandas as pd
import numpy as np

In [ ]:
filename = list(uploaded.keys())[0]
print("✅ Uploaded file:", filename)

# Load dataset
df_preview = pd.read_csv(filename)
print("Shape:", df_preview.shape)
df_preview.head()

✅ Uploaded file: Personal_Finance_Dataset (1).csv
Shape: (1500, 5)


,Date,Transaction Description,Category,Amount,Type
0,2020-01-02,Score each.,Food & Drink,1485.69,Expense
1,2020-01-02,Quality throughout.,Utilities,1475.58,Expense
2,2020-01-04,Instead ahead despite measure ago.,Rent,1185.08,Expense
3,2020-01-05,Information last everything thank serve.,Investment,2291.00,Income
4,2020-01-13,Future choice whatever from.,Food & Drink,1126.88,Expense


In [ ]:
from datetime import datetime
CSV_PATH=Path(filename)
OUTPUT_DIR=Path("/content/expense_analysis_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

In [ ]:
DATE_COLS = ["date","transaction_date","txn_date","posted_date"]
AMOUNT_COLS = ["amount","amt","value","transaction_amount","debit","credit"]
CATEGORY_COLS = ["category","cat","expense_category","merchant_category"]
DESC_COLS = ["description","memo","narration","details"]
TYPE_COLS = ["type","transaction_type","kind"]

In [ ]:
_amount_re = re.compile(r"[^\d\.\-\+]")

In [ ]:
def clean_amount(x):
    if pd.isna(x): return None
    if isinstance(x,(int,float,np.number)): return float(x)
    s=str(x).strip()
    if s.startswith("(") and s.endswith(")"):
        s="-"+s[1:-1]
    s=_amount_re.sub("",s)
    try: return float(s)
    except: return None

In [ ]:
def guess_col(candidates, columns):
    cols_lower = {c.lower(): c for c in columns}
    for name in candidates:
        if name.lower() in cols_lower: return cols_lower[name.lower()]
    return None

df = pd.read_csv(CSV_PATH, low_memory=False)
cols = df.columns

DATE_COL = guess_col(DATE_COLS, cols)
AMOUNT_COL = guess_col(AMOUNT_COLS, cols)
CATEGORY_COL = guess_col(CATEGORY_COLS, cols)
DESC_COL = guess_col(DESC_COLS, cols)
TYPE_COL = guess_col(TYPE_COLS, cols)

print(f"Detected → Date:{DATE_COL}  Amount:{AMOUNT_COL}  Category:{CATEGORY_COL}")

df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
df = df.dropna(subset=[DATE_COL])

df["Amount"] = df[AMOUNT_COL].map(clean_amount)
df = df.dropna(subset=["Amount"])

df["Category"] = df[CATEGORY_COL].fillna("Uncategorized") if CATEGORY_COL else "Uncategorized"
df["Description"] = df[DESC_COL] if DESC_COL else ""
df["Type"] = df[TYPE_COL].astype(str).str.lower() if TYPE_COL else ""

df["YearMonth"] = df[DATE_COL].dt.to_period("M").astype(str)
df["Day"] = df[DATE_COL].dt.date

Detected → Date:Date  Amount:Amount  Category:Category


In [ ]:
nonzero = df[df["Amount"]!=0]["Amount"]
neg_frac = (nonzero<0).mean() if len(nonzero)>0 else 0

if neg_frac>=0.5:
    df["Expense"] = df["Amount"].apply(lambda x:-x if x<0 else 0)
    df["Income"] = df["Amount"].apply(lambda x:x if x>0 else 0)
    sign_rule="expenses_are_negative"
else:
    if TYPE_COL:
        t=df["Type"]
        exp_mask=t.isin(["expense","debit","withdrawal","payment","spent"])
        df["Expense"]=df["Amount"].where(exp_mask,0).abs()
        df["Income"]=df["Amount"].where(~exp_mask,0).abs()
    else:
        df["Expense"]=df["Amount"].apply(lambda x:x if x>0 else 0)
        df["Income"]=df["Amount"].apply(lambda x:-x if x<0 else 0)
    sign_rule="expenses_are_positive_or_type_based"

print("Sign inference:", sign_rule)

Sign inference: expenses_are_positive_or_type_based


In [ ]:
expenses = df[df["Expense"]>0].copy()

monthly_totals = (
    df.groupby("YearMonth")
      .agg(Total_Expense=("Expense","sum"),
           Total_Income=("Income","sum"),
           Transactions=("Amount","count"))
      .reset_index()
)

cat_summary = (
    expenses.groupby("Category")
            .agg(Total=("Expense","sum"),
                 Count=("Expense","count"),
                 Avg=("Expense","mean"),
                 Median=("Expense","median"))
            .reset_index()
            .sort_values("Total", ascending=False)
)
cat_summary["Percent_of_Total"] = (cat_summary["Total"]/cat_summary["Total"].sum()*100).round(2)

peak_days = (
    expenses.groupby("Day")["Expense"].sum().reset_index()
            .sort_values("Expense",ascending=False)
)
peak_days_top = peak_days.head(5)

peak_per_month = (
    expenses.groupby(["YearMonth","Day"])["Expense"].sum().reset_index()
)
peak_per_month = peak_per_month.loc[
    peak_per_month.groupby("YearMonth")["Expense"].idxmax()
].rename(columns={"Day":"Peak_Spending_Day","Expense":"Peak_Total"})

print("✅ Monthly Totals")
display(monthly_totals.head())
print("\n✅ Category Summary")
display(cat_summary.head())
print("\n✅ Peak Days")
display(peak_days_top)

✅ Monthly Totals


,YearMonth,Total_Expense,Total_Income,Transactions
0,2020-01,17138.25,5578.0,17
1,2020-02,17108.41,20070.0,22
2,2020-03,13581.81,3465.0,17
3,2020-04,16233.05,7370.0,17
4,2020-05,16846.13,6008.0,24



✅ Category Summary


,Category,Total,Count,Avg,Median,Percent_of_Total
6,Travel,169497.79,160,1059.361188,1132.88,13.81
3,Rent,162075.39,165,982.275091,918.36,13.21
1,Food & Drink,159493.39,149,1070.425436,1162.57,13.00
4,Salary,149053.55,146,1020.914726,1082.96,12.15
0,Entertainment,148165.47,143,1036.122168,1005.52,12.07



✅ Peak Days


,Day,Expense
729,2024-01-19,5775.57
801,2024-06-23,5634.35
550,2023-01-19,5400.24
629,2023-06-25,4764.49
740,2024-02-10,4728.86


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("35KueGADcf95Z3RT8YYZ5KVaWTl_NbbBqKZ8wzPkNaN27Hvi")
print("✅ Ngrok authtoken configured successfully.")

✅ Ngrok authtoken configured successfully.


In [ ]:
import pandas as pd
from flask import Flask, jsonify, render_template_string
from pyngrok import ngrok

# 🧹 Data preprocessing
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
date_col = next((c for c in df.columns if 'date' in c), None)
amt_col  = next((c for c in df.columns if 'amount' in c or 'expense' in c), None)
cat_col  = next((c for c in df.columns if 'cat' in c or 'type' in c), None)
if not (date_col and amt_col and cat_col):
    raise ValueError("Dataset must include columns for Date, Amount, and Category.")

df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
df = df.dropna(subset=[date_col, amt_col, cat_col])
df['month'] = df[date_col].dt.to_period('M').astype(str)

monthly_totals  = df.groupby('month')[amt_col].sum().reset_index()
category_summary = df.groupby(cat_col)[amt_col].sum().reset_index().rename(columns={cat_col:'category'})
daily_summary = df.groupby(df[date_col].dt.date)[amt_col].sum().reset_index().rename(columns={date_col:'date'})
peak_days = daily_summary.sort_values('amount', ascending=False).head(5)

total_spend = df[amt_col].sum()
top_cat = category_summary.loc[category_summary['amount'].idxmax()]
peak_day = peak_days.iloc[0]

# 🌐 Flask App
app = Flask(__name__)

dashboard_html = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>💰 Expense Insights Dashboard</title>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<style>
body {
  font-family: 'Poppins', sans-serif;
  background: linear-gradient(135deg,#f0f4ff,#e8f3fa);
  margin: 0;
  color: #1e293b;
}
header {
  background: rgba(37,99,235,0.9);
  color: white;
  text-align: center;
  padding: 25px;
  font-size: 1.8em;
  letter-spacing: 0.5px;
  box-shadow: 0 2px 12px rgba(0,0,0,0.1);
}
.cards {
  display: flex;
  flex-wrap: wrap;
  justify-content: center;
  gap: 20px;
  margin: 30px auto;
  max-width: 1100px;
}
.card {
  background: rgba(255,255,255,0.7);
  backdrop-filter: blur(10px);
  border-radius: 20px;
  box-shadow: 0 8px 20px rgba(0,0,0,0.1);
  padding: 25px;
  min-width: 280px;
  text-align: center;
  transition: transform 0.3s ease;
}
.card:hover {
  transform: translateY(-6px);
}
.card h3 {
  margin: 0;
  color: #0f172a;
}
.card p {
  font-size: 1.4em;
  color: #2563eb;
  margin: 5px 0 0;
  font-weight: bold;
}
.grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(480px, 1fr));
  gap: 25px;
  padding: 0 30px 40px;
  max-width: 1200px;
  margin: 0 auto;
}
.plot-container {
  background: white;
  border-radius: 16px;
  box-shadow: 0 6px 14px rgba(0,0,0,0.08);
  padding: 20px;
  transition: transform 0.3s ease;
}
.plot-container:hover {
  transform: scale(1.02);
}
h2 {
  color: #1e3a8a;
  font-weight: 600;
  margin-bottom: 10px;
}
</style>
</head>
<body>
<header>💰 Expense Insights Dashboard</header>

<div class="cards">
  <div class="card"><h3>Total Spending</h3><p id="totalSpend"></p></div>
  <div class="card"><h3>Top Category</h3><p id="topCategory"></p></div>
  <div class="card"><h3>Peak Spending Day</h3><p id="peakDay"></p></div>
</div>

<div class="grid">
  <div class="plot-container"><h2>📅 Monthly Trends</h2><div id="monthly"></div></div>
  <div class="plot-container"><h2>🏷️ Category Breakdown</h2><div id="category"></div></div>
  <div class="plot-container"><h2>📊 Peak Spending Days</h2><div id="peak"></div></div>
</div>

<script>
async function loadData(){
  const res = await fetch('/analysis');
  const data = await res.json();

  document.getElementById('totalSpend').textContent = '$' + data.summary.total.toLocaleString();
  document.getElementById('topCategory').textContent = data.summary.top_category + ' ($' + data.summary.top_amount.toLocaleString() + ')';
  document.getElementById('peakDay').textContent = data.summary.peak_day + ' ($' + data.summary.peak_amount.toLocaleString() + ')';

  Plotly.newPlot('monthly', [{
    x: data.monthly.map(x=>x.month),
    y: data.monthly.map(x=>x.amount),
    type: 'bar',
    marker:{color:'#60a5fa'}
  }], {margin:{t:30},xaxis:{title:'Month'},yaxis:{title:'Amount ($)'}});

  Plotly.newPlot('category', [{
    labels: data.category.map(x=>x.category),
    values: data.category.map(x=>x.amount),
    type: 'pie',
    textinfo: 'label+percent',
    insidetextorientation: 'radial'
  }], {margin:{t:30}});

  Plotly.newPlot('peak', [{
    x: data.peak.map(x=>x.date),
    y: data.peak.map(x=>x.amount),
    mode: 'lines+markers',
    line:{color:'#ef4444',width:3},
    marker:{size:8}
  }], {margin:{t:30},xaxis:{title:'Date'},yaxis:{title:'Amount ($)'}});
}
loadData();
</script>
</body>
</html>
"""

@app.route('/')
def home():
    return "<h2>✅ Expense Analysis Module (Phase-3)</h2><p>Dashboard → <a href='/dashboard'>Open Interactive Dashboard</a></p>"

@app.route('/dashboard')
def dashboard():
    return render_template_string(dashboard_html)

@app.route('/analysis')
def analysis():
    return jsonify({
        'monthly': monthly_totals.to_dict(orient='records'),
        'category': category_summary.to_dict(orient='records'),
        'peak': peak_days.to_dict(orient='records'),
        'summary': {
            'total': round(float(total_spend),2),
            'top_category': top_cat['category'],
            'top_amount': round(float(top_cat['amount']),2),
            'peak_day': str(peak_day['date']),
            'peak_amount': round(float(peak_day['amount']),2)
        }
    })

# 🚀 Launch app
public_url = ngrok.connect(5000).public_url
print("🌍 Public URL:", public_url)
app.run(port=5000)


🌍 Public URL: https://waggly-bristol-swirly.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 16:47:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 16:47:06] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 16:47:07] "GET /analysis HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 16:49:30] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 16:49:30] "GET /analysis HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 16:56:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 16:56:06] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 16:56:07] "GET /analysis HTTP/1.1" 200 -
